In [8]:
import cv2
import numpy as np
import mediapipe as mp

from mediapipe.python._framework_bindings.calculator_graph import CalculatorGraph
from mediapipe.python._framework_bindings.calculator_graph import GraphInputStreamAddMode
from mediapipe.python._framework_bindings.validated_graph_config import ValidatedGraphConfig
import mediapipe.python.packet_creator
import mediapipe.python.packet_getter


# Declaring the Pose recognition, which will detect all the possible poses when angle is detected
mp_pose = mp.solutions.pose

# Drawing all landmarks
mp_drawing = mp.solutions.drawing_utils

In [9]:
# Video Characteristics
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
fps = int(cap.get(cv2.CAP_PROP_FPS))




In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b)# Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 
cap = cv2.VideoCapture('KneeBendVideo.mp4')


In [11]:
# Counter and State Variables
relax_counter = 0 
bent_counter = 0
counter = 0
stage = None
feedback = None
images_array=[]

In [12]:
#checking the landmarks
len(landmarks)

33

In [15]:
#required landmarks
hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

In [22]:
hip, knee, ankle

([0.656053900718689, 0.8197669982910156],
 [0.5185306072235107, 0.8460786938667297],
 [0.41671764850616455, 0.8673246502876282])

In [23]:
calculate_angle(hip, knee, ankle)

179.04411830446392

In [45]:
tuple(np.multiply(shoulder, [640, 480]).astype(int))

(399, 287)

In [83]:
#taking video as input
cap = cv2.VideoCapture('KneeBendVideo.mp4')

# Counter and State Variables
relax_counter = 0 
bent_counter = 0
counter = 0
stage = None
feedback = None
images_array=[]

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates of interested landmarks (23, 25, and 27)
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            # Calculating Angle
            angle = calculate_angle(hip, knee, ankle)

            # Render Detections
            a0 = int(ankle[0] * width)
            a1 = int(ankle[1] * height)

            k0 = int(knee[0] * width)
            k1 = int(knee[1] * height)

            h0 = int(hip[0] * width)
            h1 = int(hip[1] * height)

            cv2.line(image, (h0, h1), (k0, k1), (255, 255, 255), 2)
            cv2.line(image, (k0, k1), (a0, a1), (255, 255, 255), 2)
            cv2.circle(image, (h0, h1), 5, (0, 0, 0), cv2.FILLED)
            cv2.circle(image, (k0, k1), 5, (0, 0, 0), cv2.FILLED)
            cv2.circle(image, (a0, a1), 5, (0, 0, 0), cv2.FILLED)       
            
            
            # Visualizing Angle
            cv2.putText(image, 'LIVE ANGLE MEASURE', [25,590], 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(angle,4)), (20, 620), 
                        cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            relax_time = (1 / fps) * relax_counter
            bent_time = (1 / fps) * bent_counter

            #Counter Logic
            if angle > 140:
                relax_counter += 1
                bent_counter = 0
                stage = "Relaxed"
                feedback = ""
            
            if angle < 140:
                relax_counter = 0
                bent_counter += 1
                stage = "Bent"
                feedback = ""
            
            # Sucessful rep
            if bent_time == 8:
                counter += 1
                feedback = 'Rep completed'
                
            elif bent_time < 8 and stage == 'Bent':
                feedback = 'Keep Your Knee Bent'
            
            else:
                feedback = ""
        except:
            pass
        
        
            
        # Setup status box
        cv2.rectangle(image, (0,0), (int(width), 60), (0,0,0), -1)
        
        # Rep data
        cv2.putText(image, 'REPS Count', (30,15), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(counter), 
                    (30,50), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'LIVE STATUS', (200,15), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (200,50), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        
        # Feedback
        cv2.putText(image, 'LIVE UPDATE', (450,15), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.putText(image, feedback, 
                    (450,50), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        
        # Bent Time
        cv2.putText(image, 'BENT TIME', (725,15), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(round(bent_time,2)), 
                    (725,50), 
                    cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, cv2.LINE_AA) 
     
        images_array.append(image) 
        cv2.imshow('KneeBendVideo', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            counter = 0
            break

cap.release()
cv2.destroyAllWindows()
              
        

In [70]:
#declaring the video format and sample output video 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter("./output_video.mp4", fourcc, 30.0, (width, height))

# Release the video writer
output_video.release()

# Generate output video
out = cv2.VideoWriter('Output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(images_array)):
    out.write(images_array[i])
out.release()